# FER and BER Simulation Example for Fixed Reconciliation Efficiency

This script simulates IR for Gaussian modulated / 4-state modulated CV-QKD system as described in [1] for fixed reconciliation efficiency $\beta$.

In [ ]:
import sys
sys.path.append('../build')

Import the modules to be used:

In [ ]:
import information_reconciliation as ir
import numpy as np
import matplotlib.pyplot as plt

def capacity(SNR):
    return 0.5 * np.log2(1 + np.pow(10, SNR/10))

Define the simulation variables

In [ ]:
N = 1e8
max_decoding_iteration = 500
d = 8 # Reconciliation dimension
beta = 0.96
SNR_min = -18.2
SNR_max = -4.6
total_error_count = 256 # Total number of frame errors

Run the simulation

In [ ]:
statistics_array = []

for SNR in np.arange(SNR_min, SNR_max, 0.2):
    statistics_total = ir.statistics()
    sigma_square = 10**(-SNR/10) #noise variance
    code_rate = beta * capacity(SNR)

    while statistics_total.get_total_frame_error_count()<total_error_count:
        #sample the transmitted states as Gaussian
        X = np.random.normal(0, 1, int(N))
        
        #sample the transmitted states as 4-state (BPSK modulated)
        #X = np.random.choice([1, -1], int(N)) 
        
        Y = X + np.sqrt(sigma_square)*np.random.normal(0, 1, int(N))

        stats = ir.reconcile(alice_states=X, bob_states= Y, rate= code_rate, noise_variance = sigma_square, NoI=max_decoding_iteration, MDR_dim= d, fast_decoding= 1, layered_decoding= 1)
        
        statistics_total = statistics_total + stats
    
    statistics_array.append(statistics_total)
    print("SNR: ", "{:.2f}".format(SNR), "dB  FER: ", "{:.5f}".format(statistics_total.get_frame_error_rate()), " BER: ", "{:.5f}".format(statistics_total.get_bit_error_rate()), " Rate: ", "{:.5f}".format(code_rate))

[1] E. E. Cil and L. Schmalen, _"An open-source library for information reconciliation in continuous-variable QKD,"_ Proc. International Conference on Quantum Cryptography (QCRYPT), Vigo, Spain, Sep. 2024, poster presentation